In [1]:
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sqlalchemy
# import h5py

In [2]:
db_connection_string = 'sqlite:///./Resources/products.db'
engine  = sqlalchemy.create_engine(db_connection_string)

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['HSFT_1_Day_Candles', 'HSFT_Info', 'Indicators', 'MSFT_1_Day_Candles', 'MSFT_1_Min_Candles', 'MSFT_Info', 'TSLA_1_Day_Candles', 'TSLA_1_Min_Candles', 'TSLA_Info']


In [3]:
indicators_df = pd.read_sql_table('Indicators', con=engine, index_col='Datetime')

---

## Prepare the data to be used on a neural network model

In [4]:
print(indicators_df.head())

                       Open    High      Low   Close  Volume  CDLDOJI  \
Datetime                                                                
2021-09-02 12:56:00  733.89  734.31  733.750  734.21   28796        0   
2021-09-02 12:57:00  734.29  734.64  734.219  734.41   34903        0   
2021-09-02 12:58:00  734.35  734.63  734.260  734.50   19890        0   
2021-09-02 12:59:00  734.60  734.77  734.510  734.56    9812      100   
2021-09-02 13:00:00  734.35  734.66  734.290  734.50   23069        0   

                     Trailing Stop  Trade Signal  Pct Change   Stop Loss  \
Datetime                                                                   
2021-09-02 12:56:00     732.988571           1.0    0.000463  732.988571   
2021-09-02 12:57:00     733.199031           1.0    0.000272  733.199031   
2021-09-02 12:58:00     733.309457           1.0    0.000123  733.309457   
2021-09-02 12:59:00     733.406281           1.0    0.000082  733.406281   
2021-09-02 13:00:00     733.3626

In [5]:
# Review the data types associated with the columns
indicators_df.dtypes

Open                   float64
High                   float64
Low                    float64
Close                  float64
Volume                   int64
CDLDOJI                  int64
Trailing Stop          float64
Trade Signal           float64
Pct Change             float64
Stop Loss              float64
CDL3WHITESOLDIERS        int64
CDLABANDONEDBABY         int64
CDLADVANCEBLOCK          int64
CDLBELTHOLD              int64
CDLBREAKAWAY             int64
CDLCLOSINGMARUBOZU       int64
CDLCONCEALBABYSWALL      int64
CDLCOUNTERATTACK         int64
CDLDARKCLOUDCOVER        int64
dtype: object

In [6]:
display(indicators_df['Trade Signal'].value_counts())

 1.0    11282
 0.0      843
-1.0      350
Name: Trade Signal, dtype: int64

### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [7]:
# Define the target set y using the IS_SUCCESSFUL column
y = indicators_df['Trade Signal']

# Display a sample of y
display(y.head())

Datetime
2021-09-02 12:56:00    1.0
2021-09-02 12:57:00    1.0
2021-09-02 12:58:00    1.0
2021-09-02 12:59:00    1.0
2021-09-02 13:00:00    1.0
Name: Trade Signal, dtype: float64

In [8]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = indicators_df.drop(columns=['Trade Signal'])

# Review the features DataFrame
display(X.head())

,Open,High,Low,Close,Volume,CDLDOJI,Trailing Stop,Pct Change,Stop Loss,CDL3WHITESOLDIERS,CDLABANDONEDBABY,CDLADVANCEBLOCK,CDLBELTHOLD,CDLBREAKAWAY,CDLCLOSINGMARUBOZU,CDLCONCEALBABYSWALL,CDLCOUNTERATTACK,CDLDARKCLOUDCOVER
Datetime,,,,,,,,,,,,,,,,,,
2021-09-02 12:56:00,733.89,734.31,733.750,734.21,28796,0,732.988571,0.000463,732.988571,0,0,0,0,0,0,0,0,0
2021-09-02 12:57:00,734.29,734.64,734.219,734.41,34903,0,733.199031,0.000272,733.199031,0,0,0,0,0,0,0,0,0
2021-09-02 12:58:00,734.35,734.63,734.260,734.50,19890,0,733.309457,0.000123,733.309457,0,0,0,0,0,0,0,0,0
2021-09-02 12:59:00,734.60,734.77,734.510,734.56,9812,100,733.406281,0.000082,733.406281,0,0,0,0,0,0,0,0,0
2021-09-02 13:00:00,734.35,734.66,734.290,734.50,23069,0,733.362619,-0.000082,733.406281,0,0,0,0,0,0,0,0,0


### Step 6: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [11]:
# Define the the number of inputs (features) to the model
number_input_features = len(list(X.columns))
# Review the number of features
number_input_features


18

In [12]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [13]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  int(round((number_input_features + number_output_neurons)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1


10

In [14]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  int(round((hidden_nodes_layer1 + number_output_neurons)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


6

In [15]:
# Create the Sequential model instance
nn = Sequential()


In [16]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))


In [17]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))


In [18]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))


In [19]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                190       
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 263
Trainable params: 263
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [20]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [21]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [22]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

98/98 [==============================] - 0s 535us/step - loss: -9.7683 - accuracy: 0.8743
Loss: -9.768328666687012, Accuracy: 0.8743187189102173


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [24]:
# Set the model's file path
file_path = Path('Resources/AlphabetSoup.h5')

# Export your model to a HDF5 file
# nn.save(file_path)


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
>
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
>
> * Add more neurons (nodes) to a hidden layer.
>
> * Add more hidden layers.
>
> * Use different activation functions for the hidden layers.
>
> * Add to or reduce the number of epochs in the training regimen.


### Alternative Model 1 - Deep learning model with 2 hidden layers

In [25]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


18


In [26]:
# Review the number of features
number_input_features

18

In [27]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [28]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 =  int(round((number_input_features + number_output_neurons_A1)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A1


10

In [29]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 =  int(round((hidden_nodes_layer1_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A1


6

In [30]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A1 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A1)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A1


4

In [31]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [32]:
# Add the first hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation="relu", input_dim=number_input_features))


In [33]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))


In [34]:
# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer3_A1, activation="relu"))


In [35]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=number_output_neurons_A1, activation="sigmoid"))


In [36]:
# Display the Sequential model summary
nn_A1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                190       
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the Sequential model
nn_A1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [38]:
# Fit the model using 50 epochs and the training data
model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=0)


### Alternative Model 2 - Deep learning model with 3 hidden layers

In [39]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
print(number_input_features)


18


In [40]:
# Review the number of features
number_input_features

18

In [41]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [42]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 =  int(round((number_input_features + number_output_neurons_A2)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A2


10

In [43]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A2 =  int(round((hidden_nodes_layer1_A2 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A2


6

In [44]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer3_A2 =  int(round((hidden_nodes_layer2_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A2


4

In [45]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer4_A2 =  int(round((hidden_nodes_layer3_A1 + number_output_neurons_A2)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer4_A2


2

In [46]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [47]:
# Add the first hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation="relu", input_dim=number_input_features))


In [48]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation="relu"))


In [49]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation="relu"))


In [50]:
# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer4_A2, activation="relu"))


In [51]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=number_output_neurons_A2, activation="sigmoid"))


In [52]:
# Display the Sequential model summary
nn_A2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                190       
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 66        
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 28        
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 3         
Total params: 297
Trainable params: 297
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Compile the Sequential model
nn_A2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [54]:
# Fit the model using 50 epochs and the training data
model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50, verbose=0)


#### Alternative Model 3 - Principal Component Analysis

In [55]:
from sklearn.decomposition import PCA


In [56]:
# Create a StandardScaler instance
pca_scaler = StandardScaler()

# Fit the scaler to the features
X_pca_scaler = pca_scaler.fit(X)

# Fit the scaler to the features
X_pca_scaled = X_pca_scaler.transform(X)


In [57]:
for n in range(2, number_input_features):
    pca = PCA(n_components=n, random_state=1)
    X_pca = pca.fit_transform(X_pca_scaled)
    exp_var = pca.explained_variance_ratio_
#     print(f'{exp_var}')
    print(f'N = {n:3}: Total explained variance: {100*exp_var.sum():.4f}%')

N =   2: Total explained variance: 41.8793%
N =   3: Total explained variance: 49.4176%
N =   4: Total explained variance: 56.1540%
N =   5: Total explained variance: 62.2239%
N =   6: Total explained variance: 68.1662%
N =   7: Total explained variance: 74.0366%
N =   8: Total explained variance: 79.8494%
N =   9: Total explained variance: 85.1969%
N =  10: Total explained variance: 90.2461%
N =  11: Total explained variance: 94.4264%
N =  12: Total explained variance: 97.7982%
N =  13: Total explained variance: 99.9872%
N =  14: Total explained variance: 99.9980%
N =  15: Total explained variance: 99.9992%
N =  16: Total explained variance: 99.9998%
N =  17: Total explained variance: 100.0000%


In [59]:
best_n = 13
pca = PCA(n_components=best_n, random_state=1)
X_pca = pca.fit_transform(X_pca_scaled)
exp_var = pca.explained_variance_ratio_
print(f'N = {best_n:3}: Total explained variance: {100*exp_var.sum():.4f}%')


N =  13: Total explained variance: 99.9872%


In [60]:
X_pca_train, X_pca_test, y_pca_train, y_pca_test = train_test_split(X_pca, y, random_state=1)

In [61]:
# Define the the number of inputs (features) to the model
number_input_features = best_n
print(number_input_features)


13


In [62]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 1

In [63]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 =  int(round((number_input_features + number_output_neurons_A3)/2, 0))
# Review the number hidden nodes in the first layer
hidden_nodes_layer1_A3


7

In [64]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A3 =  int(round((hidden_nodes_layer1_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A3


4

In [65]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A3 =  int(round((hidden_nodes_layer2_A3 + number_output_neurons_A3)/2, 0))

# Review the number hidden nodes in the third layer
hidden_nodes_layer3_A3


2

In [66]:
# Create the Sequential model instance
nn_A3 = Sequential()

In [67]:
# Add the first hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer1_A3, activation="relu", input_dim=number_input_features))


In [68]:
# Add the second hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer2_A3, activation="relu"))


In [69]:
# Add the third hidden layer
nn_A3.add(Dense(units=hidden_nodes_layer3_A3, activation="relu"))


In [70]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A3.add(Dense(units=number_output_neurons_A3, activation="sigmoid"))


In [71]:
# Display the Sequential model summary
nn_A3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 7)                 98        
_________________________________________________________________
dense_13 (Dense)             (None, 4)                 32        
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 10        
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 3         
Total params: 143
Trainable params: 143
Non-trainable params: 0
_________________________________________________________________


In [72]:
# Compile the Sequential model
nn_A3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [73]:
# Fit the model using 50 epochs and the training data
model_A3 = nn_A3.fit(X_pca_train, y_pca_train, epochs=50, verbose=1)


Epoch 1/50
293/293 [==============================] - 0s 492us/step - loss: 0.6710 - accuracy: 0.7680
Epoch 2/50
293/293 [==============================] - 0s 479us/step - loss: 0.5582 - accuracy: 0.9105
Epoch 3/50
293/293 [==============================] - 0s 492us/step - loss: 0.4704 - accuracy: 0.9158
Epoch 4/50
293/293 [==============================] - 0s 492us/step - loss: 0.4065 - accuracy: 0.9162
Epoch 5/50
293/293 [==============================] - 0s 479us/step - loss: 0.3592 - accuracy: 0.9166
Epoch 6/50
293/293 [==============================] - 0s 479us/step - loss: 0.3167 - accuracy: 0.9163
Epoch 7/50
293/293 [==============================] - 0s 479us/step - loss: 0.2775 - accuracy: 0.9164
Epoch 8/50
293/293 [==============================] - 0s 479us/step - loss: 0.2381 - accuracy: 0.9140
Epoch 9/50
293/293 [==============================] - 0s 479us/step - loss: 0.1870 - accuracy: 0.9141
Epoch 10/50
293/293 [==============================] - 0s 479us/step - loss: 0.131

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [74]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
98/98 [==============================] - 0s 577us/step - loss: -9.7683 - accuracy: 0.8743
Loss: -9.768328666687012, Accuracy: 0.8743187189102173


In [75]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
98/98 [==============================] - 0s 412us/step - loss: -66.4234 - accuracy: 0.8798
Loss: -66.42341613769531, Accuracy: 0.8797691464424133


In [76]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
98/98 [==============================] - 0s 494us/step - loss: -2303.7139 - accuracy: 0.9186
Loss: -2303.7138671875, Accuracy: 0.9185636639595032


In [77]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_pca_test, y_pca_test, verbose=True)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
98/98 [==============================] - 0s 412us/step - loss: -83.9455 - accuracy: 0.9179
Loss: -83.94551849365234, Accuracy: 0.9179224371910095


### Run different numbers of epochs on best model

In [78]:
# Fit the model using the training data with different numbers of epochs
model_list = []
for n in range(50, 151, 20):
    print(f'Fitting model, epochs: {n}')
    model_list.append(nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False))
#     model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50, verbose=False)

    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
    model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

Fitting model, epochs: 50
98/98 [==============================] - 0s 370us/step - loss: -766.6809 - accuracy: 0.9170
Loss: -766.680908203125, Accuracy: 0.9169605374336243
Fitting model, epochs: 70
98/98 [==============================] - 0s 494us/step - loss: -3267.9072 - accuracy: 0.8221
Loss: -3267.9072265625, Accuracy: 0.822058379650116
Fitting model, epochs: 90
98/98 [==============================] - 0s 494us/step - loss: -9308.6787 - accuracy: 0.8958
Loss: -9308.6787109375, Accuracy: 0.8957999348640442
Fitting model, epochs: 110
98/98 [==============================] - 0s 494us/step - loss: -21311.8750 - accuracy: 0.8987
Loss: -21311.875, Accuracy: 0.8986854553222656
Fitting model, epochs: 130
98/98 [==============================] - 0s 495us/step - loss: -41968.5234 - accuracy: 0.9118
Loss: -41968.5234375, Accuracy: 0.911830723285675
Fitting model, epochs: 150
98/98 [==============================] - 0s 535us/step - loss: -74977.5312 - accuracy: 0.8817
Loss: -74977.53125, Accur

In [79]:
# for model in model_list:
#     pass
#     print(model.history)
    # Evaluate the model loss and accuracy metrics using the evaluate method and the test data
#     model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=True)
    # Display the model loss and accuracy results
#     print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")    

### Step 3: Save each of your alternative models as an HDF5 file.


In [81]:
# Set the file path for the first alternative model
file_path_A1 = Path('Resources/AlphabetSoup_A1.h5')

# Export your model to a HDF5 file
# nn_A1.save(file_path_A1)


In [82]:
# Set the file path for the second alternative model
file_path_A2 = Path('Resources/AlphabetSoup_A2.h5')

# Export your model to a HDF5 file
# nn_A2.save(file_path_A2)


In [83]:
# # Set the file path for the third alternative model
file_path_A3 = Path('Resources/AlphabetSoup_A3.h5')

# # Export your model to a HDF5 file
# nn_A3.save(file_path_A3)
